## <center> Portfolio Optimization-Assignment 3
#### <center>Author: Raffi Sahakyan
##### <center>Class: 19F-MGMTMSA403-1 Optimization
##### <center>Date: 6 Dec, 2019

In [1]:
# Import necessary starting libraries
from gurobipy import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Defining Optimize and Update Function for Code Efficiency
def grb_optimize(model):
    model.update()
    model.optimize()

In [2]:
#Uploading data
df_train = pd.read_csv('AirbnbTrain.csv')
df_test = pd.read_csv('AirbnbTest.csv')

df_train.head(3)

,latitude,longitude,Entire home,accommodates,bathrooms,bedrooms,beds,cleaning_fee,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,price
0,34.103701,-118.332241,1,13,2.0,3,2.0,150,2,1,100,1,350
1,34.099484,-118.331645,1,8,2.0,2,4.0,150,1,11,96,1,190
2,34.104321,-118.329662,1,4,1.0,0,1.0,55,1,1,80,0,85


#### __Model 1__ 
<br>
Formulate the least absolute deviations regression problem as a linear program. Solve the linear program using the data given in the file AirbnbTrain.csv. What is the prediction error, in $/night, of your model on the test set (provided in AirbnbTest.csv)?

In [3]:
#Initializing the model
model1 = Model()

#Adding Variables
columns = df_train.shape[1]-1 #-1 to exclude the y variable
rows = df_train.shape[0]
y_train = df_train.price
x_train = df_train.drop('price',axis=1)

z=model1.addVars(rows)
b=model1.addVars(columns)


#Adding Constraints
for row in range(rows):
    model1.addConstr(z[row]>= y_train[row] - sum(np.multiply(b[col], x_train.iloc[row, col]) for col in range(columns)))

for row in range(row):
    model1.addConstr(z[row]>= -y_train[row] + sum(np.multiply(b[col], x_train.iloc[row, col])  for col in range(columns)))
    
#Setting objective
model1.setObjective(sum(z[row] for row in range(rows))/rows, GRB.MINIMIZE)
    
#Gurobi optimize
grb_optimize(model1)

Academic license - for non-commercial use only
Optimize a model with 3399 rows, 1712 columns and 41357 nonzeros
Coefficient statistics:
  Matrix range     [5e-01, 5e+02]
  Objective range  [6e-04, 6e-04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.08s
Presolved: 3399 rows, 1712 columns, 41357 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 12
 AA' NZ     : 3.966e+04
 Factor NZ  : 4.313e+04 (roughly 2 MBytes of memory)
 Factor Ops : 5.490e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.12302582e+03  0.00000000e+00  1.36e+03 0.00e+00  2.56e+01     0s
   1   2.05670711e+03 -1.60039401e+00  0.00e+00 5.43e-03  1.75e+00     0s
   2   1.14596696e+03  2.86526140e+00  0.00e+00 7.01e-05  2.34e-01     0s
   3   1.134966

In [4]:
y = df_test.price
x = df_test.drop('price', axis=1)

cols_test = df_test.shape[1] - 1
rows_test = df_test.shape[0]

In [5]:
initial_error = 0
for row in range(rows_test):
    initial_error+= abs(y[row]- sum(b[col].x * x.iloc[row,col] for col in range(cols_test)))
error = initial_error/rows_test

In [6]:
print("The prediction error is $/night: ", round(error,3))

The prediction error is $/night:  35.627


__Model 2__ Suppose that to improve interpretability, you wish to build a model that predicts Airbnb prices using only the three most important variables. Modify Model 1 by including a constraint that allows at most three variables to have non-zero coefficients.

In [7]:
#Additional large constant
bbar = 1000

#Additional Variables
v = model1.addVars(columns, vtype = GRB.BINARY)

#Additional Constraints
for col in range(columns):
    model1.addConstr(b[col]<=bbar * v[col])

for col in range(columns):
    model1.addConstr(b[col]>=-bbar * v[col])    
    
model1.addConstr(sum(v[col] for col in range(columns))<=3)

grb_optimize(model1)

Optimize a model with 3424 rows, 1724 columns and 41417 nonzeros
Variable types: 1712 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+03]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+03]
Found heuristic solution: objective 144.9682353
Presolve removed 836 rows and 412 columns
Presolve time: 0.04s
Presolved: 2588 rows, 1312 columns, 31305 nonzeros
Variable types: 1300 continuous, 12 integer (12 binary)

Root relaxation: objective 3.644463e+01, 1444 iterations, 0.21 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.44463    0    7  144.96824   36.44463  74.9%     -    0s
H    0     0                      48.4678855   36.44463  24.8%     -    0s
     0     0   36.44464    0   10   48.46789   36.44464  24.8%     -    0s
H    0     0                      47.6738244   36.44

In [8]:
initial_error = 0
for row in range(rows_test):
    initial_error += abs(y[row]- sum(b[col].x * x.iloc[row, col] for col in range(cols_test)))
new_error= initial_error/row

index_b = []
for index in range(len(b)):
    if (b[index].getAttr(GRB.Attr.X)) != 0:
         index_b.append(index)

* List the names and coefficients of the three variables selected by the optimization model. 
* What is the new prediction error, in $/night, of Model 2?

In [9]:
coef_df=pd.DataFrame(data=[b[i].getAttr(GRB.Attr.X) for i in index_b], index=df_train.columns[index_b], columns=['Coefficients'])
coef_df.index.name = 'Names'
coef_df

,Coefficients
Names,
Entire home,52.0
accommodates,14.0
bedrooms,32.0


In [10]:
print("New prediction error is $/night: ", round(new_error,3))

New prediction error is $/night:  37.791


__Model 3__. Suppose now you wish to build a model that predicts Airbnb listing price using only three variables, where one of the variables is the number of beds.

In [11]:
#Additional constraints
model1.addConstr(v[6] == 1) #6 is the index of column beds
grb_optimize(model1)

Optimize a model with 3425 rows, 1724 columns and 41418 nonzeros
Variable types: 1712 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+03]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]

MIP start did not produce a new incumbent solution
MIP start violates constraint R3424 by 1.000000000

Found heuristic solution: objective 144.9682353
Presolve removed 838 rows and 413 columns
Presolve time: 0.04s
Presolved: 2587 rows, 1311 columns, 31302 nonzeros
Variable types: 1300 continuous, 11 integer (11 binary)

Root relaxation: objective 3.644463e+01, 1464 iterations, 0.27 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.44463    0    8  144.96824   36.44463  74.9%     -    0s
H    0     0                      48.6725044   36.44463  25.1%     -    0s
     0     0   36.45476

In [12]:
intitial_error = 0
for row in range(rows_test):
    intitial_error+= abs(y[row]- sum(b[col].x*x.iloc[row, col] for col in range(cols_test)))
new_error = intitial_error/rows_test

index_b1= []
for bi in range(len(b)):
    if (b[bi].getAttr(GRB.Attr.X)) != 0:
         index_b1.append(bi)

* List the names and coefficients of the two other variables selected by the optimization model.
* Which variable was in Model 2 but is no longer in Model 3? Briefly explain in 1-2 sentences why this variable might have been dropped.
* What is the new prediction error, in $/night, of Model 3?

In [13]:
coef_df_new=pd.DataFrame(data=[b[i].getAttr(GRB.Attr.X) for i in index_b1], index=df_train.columns[index_b1], columns=['Coefficients'])
coef_df_new.index.name = 'Names'
coef_df_new

,Coefficients
Names,
Entire home,67.875
bedrooms,47.375
beds,12.125


In [14]:
print("The dropped variable is 'Accomodates'")
print("New prediction error is $/night: ", round(new_error,3))

The dropped variable is 'Accomodates'
New prediction error is $/night:  38.6


_Answer 3b:_ Variable "Accomodates" is correlated with the variable "beds", so variability in dependent variable can be equally highly explained by variabiliy in "beds". For the latter reason, the variable accomodates was dropped. To prove the latter statement we have to check the pearson correlation coefficient.

In [19]:
from scipy.stats.stats import pearsonr
print("Correllation is: ",round(pearsonr(df_train.dropna().accommodates, df_train.dropna().beds)[0],3))

Correllation is:  0.715
